In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline 

# ME 565, 02/17/2023

### Anastasia Bizyaeva

We are looking to numerically solve the 2D Poisson equation $\nabla^{2} u =  f(x,y)$ using central differences,  (1) on a rectangular domain and (2) on a circular domain 

## (1) Poisson equation on a rectangle

$\nabla^{2} u = u_{xx} + u_{yy} = f(x,y)$

$x \in [0,L_x]$ 

$y \in [0,L_y]$ 

Dirichlet boundary conditions: 

Left: $u(0,y) = f_1(y)$

Right: $u(L_x,y) = f_2(y)$

Lower: $u(x,0) = f_3(y)$

Upper: $u(x,L_y) = f_4(y)$


In [ ]:
# First generate and the grid points 

Nx = 50           # Number of grid points in x 
Ny = 50           # Number of grid points in y
Lx = 1.0          # Length of the domain in x
Ly = 1.0          # Length of the domain in y
x=np.linspace(0,Lx,Nx)
dx = x[1]-x[0]    # Grid spacing in x
y=np.linspace(0,Ly,Ny)
dy = y[1]-y[0]    # Grid spacing in y
X, Y = np.meshgrid(x, y) # Generate the cooridnate grid 

# Visualize

fig = plt.figure()
plt.plot(x[1],y[1],'ks',label='Interior Point')
plt.plot(X,Y,'ks')
plt.plot(Ly*np.ones(Ny),y,'rs',label='Boundary Point')
plt.plot(np.zeros(Ny),y,'rs')
plt.plot(x,np.zeros(Nx),'rs')
plt.plot(x, Lx*np.ones(Nx),'rs')
plt.xlim((-0.1,Lx+0.1))
plt.ylim((-0.1,Ly+0.1))
plt.xlabel('x')
plt.ylabel('y')
plt.gca().set_aspect('equal', adjustable='box')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title(r'Grid $\Delta x$ = '+str((round(dx,2)))+', $\Delta y$ = '+str((round(dy,2))),fontsize=18)

Recall that second derivative of $u(x,y)$ can be approximated using central differences: 

$u_{xx}(x_i,y_j) \approx \frac{1}{(\Delta x)^2} \big( u(x_{i-1},y_j) - 2 u(x_i,y_j) + u(x_{j+1},y_j)\big)$

$u_{yy}(x_i,y_j) \approx \frac{1}{(\Delta y)^2} \big( u(x_{i},y_{j-1}) - 2 u(x_i,y_j) + u(x_{j},y_{j+1})\big)$

In [ ]:
# Build a few boundary conditions


# u_b1: Dirichlet boundary conditions with sinusoids on all boundaries
u_b1 =np.zeros((Nx,Ny))  # Placeholder for oundary values of u

for i in range (0,Nx):
        u_b1[i,0]=np.sin(2*np.pi*x[i])       # Lower Boundary Condition
        u_b1[i,Ny-1]=np.sin(2*np.pi*x[i])    # Upper Boundary Condition

for j in range (0,Ny):
        u_b1[0,j]=2*np.sin(2*np.pi*y[j])     # Left Boundary Condition
        u_b1[Nx-1,j]=2*np.sin(2*np.pi*y[j])  # Right Boundary Condition 


# u_b2: Dirichlet boundary conditions with a parabola on the right boundary and zero on all other boundaries
u_b2 =np.zeros((Nx,Ny))  # Placeholder for oundary values of u

for i in range (0,Nx):
        u_b2[i,0]=0                                   # Lower Boundary Condition
        u_b2[i,Ny-1]=0                                # Upper Boundary Condition

for j in range (0,Ny):
        u_b2[0,j]=0                                   # Left Boundary Condition
        u_b2[Nx-1,j]= -(4/Ly**2)*(y[j] - Ly/2)**2+1   # Right Boundary Condition 

In [ ]:
# select boundary condition

u_b = u_b2

# plotitng options 
%matplotlib inline 
#%matplotlib nbagg

# Visualize boundary conditions         
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X, Y, u_b.T, rstride=Ny, cstride=Nx)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('$u_b(x,y)$')
plt.title(r'Boundary Conditions',fontsize=20)
plt.show()

We want to solve a linear algebra problem of the form 

$ A \mathbf{u} = \mathbf{f} $

where $A$ is a $N_x N_y \times N_x N_y$ matrix, and $\mathbf{u}, \mathbf{f}$ are $N_x N_y$ element vectors.

In [ ]:
A=np.zeros((Nx*Ny,Nx*Ny))

# Build the differential operator matrix row by row for interior points       
Nint = 0
for i in range (1,Nx-1):
    for j in range (1,Ny-1):    
        Nint = Nint + 1
        A[i*Ny+ j , (i-1)*Ny + j] = (1/dx**2)
        A[i*Ny + j , i*Ny + j - 1] = (1/dy**2)
        A[i*Ny + j , i*Ny + j] = -2*(1/dx**2 + 1/dy**2)
        A[i*Ny + j , i*Ny + j + 1]=(1/dy**2)
        A[i*Ny + j , (i+1)*Ny + j ]=(1/dx**2)
        

# Fill ones on the diagonal for boundary points 
NBC = 0
for i in range (0,Nx):
    for j in range (0,Ny):
        if i == 0 or j == 0 or i == Nx-1 or j == Ny-1:
            NBC = NBC + 1
            A[i*Ny + j , i*Ny + j ]=1

# Visualize the matrix A

fig = plt.figure(figsize=(10,4))
plt.imshow(A,interpolation='none')
clb=plt.colorbar()
clb.set_label('Matrix element values')
plt.title('Matrix $A$, $N_x = $'+str(Nx)+', $N_y = $'+str(Ny),fontsize=18)
fig.tight_layout()
plt.show()

In [ ]:
# Build the right hand side vector f of inhomogeneity and boundary conditions

def f(x,y):  # Source term in the PDE - 0 for Laplace eqn, f(x,y) for Poisson eqn - change this to solve different problems! 
    return 0 # x**2 + y**2

fvec = np.zeros(Nx*Ny)

# Fill in the inhomogeneity
for i in range (1,Nx-1):
    for j in range (1,Ny-1):
        fvec[i*Ny + j] = f(x[i],y[j])

# Fill in the boundary conditions
for i in range (0,Nx):
    for j in range (0,Ny):
        if i == 0 or j == 0 or i == Nx-1 or j == Ny-1:
            fvec[i*Ny + j] = u_b[i,j]

# Solve the linear system
u = np.linalg.solve(A,fvec)

# Reshape the solution vector u into a matrix 
u = np.reshape(u,(Nx,Ny)).T # transpose because of the way python fills matrices row by row

# Visualize the solution as a wireframe plot 
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(X, Y, u)#, color='r', rstride=10, cstride=10)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('$u$')
plt.title(r'$u(x,y)$',fontsize=20,y=1.08)
plt.show()

In [ ]:
# visualize the solution u(x,y) as a surface plot in 3d

fig2 = plt.figure(figsize=(6,6))
ax2 = plt.axes(projection='3d')             # initialize 3d figure
ax2.view_init(30,45)                        # set figure viewing angle (elevation, azimuth in degrees)

surf = ax2.plot_surface(X, Y,u, cmap='plasma', edgecolor='none')

# generate a colorbar
cbar = fig2.colorbar(surf,shrink=0.7, aspect=15, pad = 0.02)

# set figure labels
ax2.set_title(r'$u(x,y)$',fontsize=24)
ax2.set_xlabel('x',fontsize=20)
ax2.set_ylabel('y',fontsize=20)
ax2.set_zlabel('u(x,y)',fontsize=20)
cbar.set_label(label='u(x,y)',fontsize = 20)


In [ ]:
# plot the function u(x,y) as a heatmap over the (x,y)-coordinate grid 

fig3,ax3 = plt.subplots(figsize=(6,6))

heatmap = plt.imshow(u,cmap='plasma', origin = 'lower',extent=[x[0],x[-1],y[0],y[-1]] )
cbar = fig3.colorbar(heatmap,pad = 0.1,shrink = 0.75)
cbar.set_label(label='u(x,y)',fontsize = 20)

ax3.set_title('Heatmap of u(x,y)',fontsize=24)
ax3.set_xlabel('x',fontsize=20)
ax3.set_ylabel('y',fontsize=20)

## (2) Challenge: Poisson equation on a circle

$\nabla^2 u(r,\theta) = \frac{1}{r} \frac{\partial}{\partial r}(r u_{r})  + \frac{1}{r} u_r +  = f(r,\theta)$ 

$r \in [0,R]$

$\theta \in [0,2 \pi]$ 

Boundary conditions: 

$u(R,\theta) = f(\theta)$, $f(\theta) = f(\theta + 2 \pi)$

$u(r,\theta) = u(r,\theta + 2\pi)$ 

In [ ]:

# First generate and the grid points 

Nr = 10                   # Number of grid points in r 
Ntheta = 10               # Number of grid points in theta
Lr = 1.0                  # Length of the domain in r
Ltheta = 2*np.pi          # Length of the domain in theta
r=np.linspace(0,Lr,Nr)
dr = r[1]-r[0]            # Grid spacing in r
theta=np.linspace(0,Ltheta,Ntheta)
dtheta = theta[1]-theta[0]       # Grid spacing in theta
R, Theta = np.meshgrid(r, theta) # Generate the cooridnate grid 

# Visualize

fig = plt.figure(figsize=(6,6))
plt.plot(r[1],theta[1],'ks',label='Interior Point')
plt.plot(R,Theta,'ks')
plt.plot(Lr*np.ones(Ntheta),theta,'rs',label='Boundary Point')
plt.plot(np.zeros(Ntheta),theta,'rs')
plt.plot(r,np.zeros(Nr),'rs')
plt.plot(r, Ltheta*np.ones(Nr),'rs')
plt.xlim((-0.1,Lr+0.1))
plt.ylim((-0.1,Ltheta+0.1))
plt.xlabel('$r$')
plt.ylabel('$\Theta$')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title(r'Grid $\Delta r$ = '+str((round(dr,2)))+', $\Delta$ $\Theta$ = '+str((round(dtheta,2))),fontsize=18)

In [ ]:
# Visualize grid points in Cartesian coordinates

X = R*np.cos(Theta)
Y = R*np.sin(Theta)

fig = plt.figure(figsize=(6,6))
plt.plot(X[1],Y[1],'ks',label='Interior Point')
plt.plot(X,Y,'ks')
plt.plot(Lr*np.cos(theta),Lr*np.sin(theta),'rs',label='Boundary Point')
plt.plot(np.zeros(Ntheta),np.zeros(Ntheta),'rs')
plt.plot(r*np.cos(0),r*np.sin(0),'rs')
plt.plot(r*np.cos(Ltheta),r*np.sin(Ltheta),'rs')
plt.xlim((-0.1-Lr,Lr+0.1))
plt.ylim((-0.1-Lr,Lr+0.1))
plt.xlabel('$x = r \cos(\Theta)$')
plt.ylabel('$y = r \sin(\Theta)$')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title(r'Grid $\Delta r$ = '+str((round(dr,2)))+', $\Delta$ $\Theta$ = '+str((round(dtheta,2))),fontsize=18)

In [2]:
# Challenge exercise: write code, solving the Poisson equation in polar coordinates using central differences over the r-theta grid
# Hint: Your A matrix will depend on the grid spacing in r and theta! 
# Hint: Make sure to include all of the derivative terms in your A matrix: in polar coordinates the Laplacian has a first-order derivative term 